# Metadata
- This notebook is for processing the various raw information scraped and combining it into one `metadata.csv`
- For each data source, the file will be stored at `data/raw/{data source}/metadata.csv`

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import Pool
import os
import requests

In [2]:
# change working directory to the project root directory
current_dir = os.getcwd()
os.chdir(current_dir + '/../../')
# this should be the project root directory
os.getcwd()

'/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage'

## INES

- Uses the existing columns in `INES_Metadata.csv`
- checks if the video file exists or not
- adds the column `file_exists`

Load the `INES_metadata.csv` to get the Video URLs

In [101]:
ines_csv = pd.read_csv('data/raw/INES/INES_Metadata.csv')
ines_csv.head()

,Letter,Word,Video URL,Image URL,Assuntos,Acepção,Exemplo,Exemplo Libras,Classe Gramatical,Origem
0,A,A,https://www.ines.gov.br/dicionario-de-libras/p...,https://www.ines.gov.br/https://www.ines.gov.b...,NENHUM,Primeira letra do alfabeto da língua portugues...,Invente qualquer palavra que comece com a letr...,VOCÊ INVENTAR QUALQUER PALAVRA COMEÇAR A.,SUBSTANTIVO,Nacional
1,A,ABACATE,https://www.ines.gov.br/dicionario-de-libras/p...,https://www.ines.gov.br/https://www.ines.gov.b...,FRUTA,"O fruto do abacateiro. Comestível, tem a polpa...",Você gosta de abacate com leite?,VOCÊ GOSTAR ABACATE LEITE JUNTO?,SUBSTANTIVO,Nacional
2,A,ABACAXI,https://www.ines.gov.br/dicionario-de-libras/p...,https://www.ines.gov.br/https://www.ines.gov.b...,FRUTA,Fruta de casca grossa e áspera. Sua polpa pode...,"Hoje tomei suco de abacaxi, ele estava ácido.",HOJE S-U-C-O ABACAXI BEBER ÁCID@.,SUBSTANTIVO,Nacional
3,A,ABAFAR,https://www.ines.gov.br/dicionario-de-libras/p...,https://www.ines.gov.br/https://www.ines.gov.b...,NENHUM,"Cobrir ou fechar, para manter o calor.","Se você quer abafar seu quarto, é melhor fecha...",S-I VOCÊ QUERER QUARTO SE@ ABAFAR A-R? MELHOR ...,VERBO,Nacional
4,A,ABAIXO,https://www.ines.gov.br/dicionario-de-libras/p...,https://www.ines.gov.br/https://www.ines.gov.b...,NENHUM,"Lugar, posição ou situação inferior, em relaçã...","Não é no primeiro apartamento abaixo, é no seg...",APARTAMENTO PRIMEIR@ NÃO SEGUND@ ABAIXO.,ADV.,Nacional


Find a difference in response between URLs for words that have/don't have a video file 

In [43]:
# invalid url
invalid = 'https://www.ines.gov.br/dicionario-de-libras/public/media/palavras/videos/baba1Sm_Prog001.mp4'
response = requests.get(invalid, stream=True)
response.headers['Content-Type']

'text/html;charset=utf-8'

In [44]:
# valid url
valid = 'https://www.ines.gov.br/dicionario-de-libras/public/media/palavras/videos/babadorSm_Prog001.mp4'
response = requests.get(valid, stream=True)
response.headers['Content-Type']

'video/mp4'

This takes a while to run since it checks every URL, so just get the info from `video_file_exists.csv` or `metadata.csv`

*(took X minutes on Ben's PC/connection)*

In [45]:
import asyncio
import aiohttp
import random
from aiohttp import ClientConnectorError, ClientTimeout

In [46]:
video_urls = ines_csv['Video URL'].tolist()

file_exists = []

In [47]:
# Limit concurrent requests
MAX_CONCURRENT = 20  # Reduce this if still getting errors
RETRY_COUNT = 5     # Increased from 3

async def check_url(url, index, total, semaphore, retry_count=RETRY_COUNT):
    timeout = ClientTimeout(total=30)
    
    async with semaphore:  # Control concurrent requests
        for attempt in range(retry_count):
            try:
                # Longer delay between requests
                await asyncio.sleep(random.uniform(0.5, 1.0))
                
                async with aiohttp.ClientSession(timeout=timeout) as session:
                    async with session.get(url) as response:
                        exists = response.headers['Content-Type'] == 'video/mp4'
                        percent = (index + 1) / total * 100
                        if percent % 5 < (1 / total * 100):
                            print(f"Progress: {percent:.1f}% ({index + 1}/{total} URLs checked)")
                        return exists
                        
            except (ClientConnectorError, asyncio.TimeoutError) as e:
                if attempt == retry_count - 1:  # Last attempt
                    print(f"\nFailed to check URL after {retry_count} attempts: {url}\nError: {str(e)}")
                    return False
                print(f"\nRetrying URL {index + 1}/{total} (attempt {attempt + 2}/{retry_count})")
                # Longer wait between retries
                await asyncio.sleep(random.uniform(2, 3))
            except Exception as e:
                print(f"\nUnexpected error for URL {index + 1}/{total}: {str(e)}")
                return False

async def check_all_urls(urls):
    total = len(urls)
    print(f"Starting to check {total} URLs...")
    
    # Create semaphore to limit concurrent requests
    semaphore = asyncio.Semaphore(MAX_CONCURRENT)
    
    tasks = [check_url(url, i, total, semaphore) for i, url in enumerate(urls)]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Filter out any exceptions and count successes
    valid_results = [r for r in results if isinstance(r, bool)]
    exists_count = sum(1 for r in valid_results if r)
    
    print(f"\nFinished checking URLs. Found {exists_count} existing videos out of {total}")
    return results

# Use it with:
file_exists = await check_all_urls(video_urls)

Starting to check 5819 URLs...
Progress: 5.0% (291/5819 URLs checked)
Progress: 10.0% (582/5819 URLs checked)
Progress: 15.0% (873/5819 URLs checked)
Progress: 20.0% (1164/5819 URLs checked)
Progress: 25.0% (1455/5819 URLs checked)
Progress: 30.0% (1746/5819 URLs checked)
Progress: 35.0% (2037/5819 URLs checked)
Progress: 40.0% (2328/5819 URLs checked)
Progress: 45.0% (2619/5819 URLs checked)
Progress: 50.0% (2910/5819 URLs checked)
Progress: 55.0% (3201/5819 URLs checked)
Progress: 60.0% (3492/5819 URLs checked)
Progress: 65.0% (3783/5819 URLs checked)
Progress: 70.0% (4074/5819 URLs checked)
Progress: 75.0% (4365/5819 URLs checked)
Progress: 80.0% (4656/5819 URLs checked)
Progress: 85.0% (4947/5819 URLs checked)

Retrying URL 4734/5819 (attempt 2/5)

Retrying URL 4827/5819 (attempt 2/5)
Progress: 90.0% (5238/5819 URLs checked)
Progress: 95.0% (5529/5819 URLs checked)
Progress: 100.0% (5819/5819 URLs checked)

Finished checking URLs. Found 5468 existing videos out of 5819


In [88]:
len(file_exists)

5819

Save this as `video_file_exists.csv` so that if `metadata.csv` is overwritten, we don't lose this info

In [89]:
file_exists_df = ines_csv[['Word', 'Video URL']].copy()
file_exists_df.rename(columns={'Word':'label','Video URL': 'video_url'}, inplace=True)
file_exists_df['file_exists'] = file_exists

file_exists_df.to_csv('data/raw/INES/video_file_exists.csv', index=False)

Make `metadata_df` using the already saved `video_file_exists.csv` to avoid running the requests code again

In [3]:
ines_csv = pd.read_csv('data/raw/INES/INES_Metadata.csv')
ines_csv.columns = ines_csv.columns.str.lower()
ines_csv.rename(columns={'word':'label', 'video url':'video_url'}, inplace=True)
ines_csv.drop(columns=['image url'], inplace=True)

In [4]:
file_exists_df = pd.read_csv('data/raw/INES/video_file_exists.csv')

In [5]:
file_exists_df.shape, ines_csv.shape

((5819, 3), (5819, 9))

In [6]:
(file_exists_df.label == ines_csv.label).sum(), (file_exists_df.video_url == ines_csv.video_url).sum()

(5819, 5819)

In [7]:
metadata_df = pd.concat([ines_csv, file_exists_df.drop(['label', 'video_url'], axis=1)], axis=1)

In [9]:
metadata_df.columns

Index(['letter', 'label', 'video_url', 'assuntos', 'acepção', 'exemplo',
       'exemplo libras', 'classe gramatical', 'origem', 'file_exists'],
      dtype='object')

In [10]:
def number_in_label(row):
    """Returns True if there is a number in the label else False"""
    return any(char.isdigit() for char in row["label"])


def remove_number_from_label(row):
    """Removes numbers from the label"""
    return "".join([char for char in row["label"] if not char.isdigit()])

In [11]:
metadata_df["number_in_label"] = metadata_df.progress_apply(number_in_label, axis=1)
metadata_df["scraped_label"] = metadata_df["label"]
metadata_df["label"] = metadata_df.progress_apply(remove_number_from_label, axis=1)

100%|██████████| 5819/5819 [00:00<00:00, 141333.82it/s]


In [12]:
metadata_df['signer_number'] = 0
metadata_df['data_source'] = 'ne'

In [14]:
# Save the metadata.csv file
metadata_df[
    [
        "label",
        "video_url",
        "signer_number",
        "data_source",
        "scraped_label",
        "number_in_label",
        "file_exists",
        "letter",
        "assuntos",
        "acepção",
        "exemplo",
        "exemplo libras",
        "classe gramatical",
        "origem",
    ]
].to_csv("data/raw/INES/metadata.csv", index=False)

In [15]:
metadata_df.file_exists.value_counts()

file_exists
True     5468
False     351
Name: count, dtype: int64

## V-Librasil

- Uses the existing information in `v_librasil_words_n_links.txt`
- Uses the links to each word's page
- Collects the video file links for each interpreter
- Combines it into a `.csv` file

#### Fixing `v_librasil_words_n_links.txt`

In [3]:
# load .txt file as a string
words_and_links = open('data/raw/V-Librasil/words/v_librasil_words_n_links.txt', 'r').read()
words_and_links = words_and_links.split('\n')

In [4]:
# one line had two lines combined
for line in words_and_links:
    if len(line.split('https://')) > 2:
        print(line)

In [5]:
# i edited the file directly to fix it
# load .txt file as a string
words_and_links = open('data/raw/V-Librasil/words/v_librasil_words_n_links.txt', 'r').read()
words_and_links = words_and_links.split('\n')

# one line had two lines combined
for line in words_and_links:
    if 'Bigode' in line:
        print(line)
    if 'Bilhão' in line:
        print(line)

Bigode https://libras.cin.ufpe.br/sign/826
Bilhão https://libras.cin.ufpe.br/sign/105


#### Making `metadata.csv`

turning `v_librasil_words_n_links.txt` into a DataFrame

In [6]:
words_and_links = pd.read_csv('data/raw/V-Librasil/words/v_librasil_words_n_links.txt', sep='https', header=None, engine='python')
words_and_links.columns = ['label', 'sign_url']
words_and_links.sign_url = words_and_links.sign_url.apply(lambda x: 'https' + x)
words_and_links.head()

,label,sign_url
0,À noite toda,https://libras.cin.ufpe.br/sign/885
1,À tarde toda,https://libras.cin.ufpe.br/sign/100
2,Abacaxi,https://libras.cin.ufpe.br/sign/817
3,Abanar,https://libras.cin.ufpe.br/sign/1536
4,Abandonar,https://libras.cin.ufpe.br/sign/71


getting video URLs (~3 for each sign)

In [7]:
from bs4 import BeautifulSoup

def get_video_urls(url):
    response = requests.get(url)

    if response.status_code == 200:
        video_links = []
        signer_numbers = []
        signer_order = ''
        html = response.content
        soup = BeautifulSoup(html, 'html.parser')
        
        # go inside div class container
        container = soup.find('section', class_='page-section').find('div', class_='container')
        # go inside div class row
        rows = [child for child in container.children if child.name == 'div']

        # get the video and signer number
        for row in rows:
            signer_number = row.find('h2').text.strip().split(' ')[1]
            link = row.find('source').get('src')
            video_links.append(link)
            signer_numbers.append(signer_number)
            signer_order += signer_number
        return video_links, signer_numbers, signer_order

    else:
        print(f"Response code != 200, Failed to get video URLs for {link}")
        return None

In [8]:
words_and_links['all_video_info'] = words_and_links['sign_url'].progress_apply(get_video_urls)

  4%|▍         | 53/1364 [00:30<12:36,  1.73it/s]


KeyboardInterrupt: 

One by one takes a while, so I asked AI to make it async

In [8]:
df = words_and_links.copy()

In [9]:
import pandas as pd
import asyncio
import aiohttp
from tqdm import tqdm
import logging
import random
from typing import Dict, Any
from bs4 import BeautifulSoup

class AsyncRequestProcessor:
    def __init__(
        self,
        max_concurrent: int = 10,
        max_retries: int = 3,
        retry_delay: float = 1.0,
        timeout: int = 30
    ):
        self.max_concurrent = max_concurrent
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.timeout = timeout
        self.setup_logging()

    def setup_logging(self):
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    async def process_url(
        self,
        url: str,
        session: aiohttp.ClientSession,
        semaphore: asyncio.Semaphore
    ) -> Dict[str, Any]:
        for attempt in range(self.max_retries):
            try:
                async with semaphore:
                    # Random delay between requests
                    await asyncio.sleep(random.uniform(0.1, 0.3))
                    
                    # Call get_video_urls function
                    video_links, signer_numbers, signer_order = await get_video_urls(url, session)
                    return {
                        'url': url,
                        'status': 'success',
                        'video_links': video_links,
                        'signer_numbers': signer_numbers,
                        'signer_order': signer_order,
                        'attempts': attempt + 1
                    }
                    
            except aiohttp.ClientError as e:
                if attempt == self.max_retries - 1:
                    return {
                        'url': url,
                        'status': 'error',
                        'error': f'Network error: {str(e)}',
                        'attempts': attempt + 1
                    }
                await asyncio.sleep(self.retry_delay * (attempt + 1))
            except Exception as e:
                return {
                    'url': url,
                    'status': 'error',
                    'error': str(e),
                    'attempts': attempt + 1
                }

    async def process_urls(self, urls: list) -> list:
        semaphore = asyncio.Semaphore(self.max_concurrent)
        timeout = aiohttp.ClientTimeout(total=self.timeout)
        
        async with aiohttp.ClientSession(timeout=timeout) as session:
            tasks = [
                self.process_url(url, session, semaphore)
                for url in urls
            ]
            
            results = []
            failed = 0
            for f in tqdm(
                asyncio.as_completed(tasks),
                total=len(tasks),
                desc="Processing URLs"
            ):
                result = await f
                if result['status'] == 'error':
                    failed += 1
                    self.logger.warning(
                        f"Failed to process {result['url']}: {result['error']}"
                    )
                results.append(result)
            
            self.logger.info(
                f"Completed processing {len(results)} URLs. "
                f"Failed: {failed}"
            )
            return results

async def get_video_urls(url: str, session: aiohttp.ClientSession):
    async with session.get(url) as response:
        if response.status == 200:
            video_links = []
            signer_numbers = []
            signer_order = ''
            
            # Get the HTML content
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')
            
            # go inside div class container
            container = soup.find('section', class_='page-section').find('div', class_='container')
            # go inside div class row
            rows = [child for child in container.children if child.name == 'div']

            # get the video and signer number
            for row in rows:
                signer_number = row.find('h2').text.strip().split(' ')[1]
                link = row.find('source').get('src')
                video_links.append(link)
                signer_numbers.append(signer_number)
                signer_order += signer_number
            return video_links, signer_numbers, signer_order
        else:
            raise aiohttp.ClientError(f"Response code {response.status}, Failed to get video URLs for {url}")

# For Jupyter notebook execution
async def main(df):
    processor = AsyncRequestProcessor(
        max_concurrent=10,
        max_retries=3,
        retry_delay=1.0,
        timeout=30
    )
    
    results = await processor.process_urls(df['sign_url'].tolist())
    return pd.DataFrame(results)

# Execute in Jupyter
try:
    # Get current notebook's event loop
    loop = asyncio.get_event_loop()
    results_df = await main(df)  # Use await here in Jupyter
    
    # Process results into final dataframe
    successful_results = results_df[results_df['status'] == 'success']
    
    # Create separate columns for video links and signer information
    df_expanded = pd.concat([
        df,
        pd.DataFrame({
            'video_links': successful_results['video_links'],
            'signer_numbers': successful_results['signer_numbers'],
            'signer_order': successful_results['signer_order']
        })
    ], axis=1)
    
except Exception as e:
    logging.error(f"Processing failed: {str(e)}")

Processing URLs: 100%|██████████| 1364/1364 [03:20<00:00,  6.79it/s]
2025-03-06 17:12:54,283 - INFO - Completed processing 1364 URLs. Failed: 0


In [10]:
results_df.head()

,url,status,video_links,signer_numbers,signer_order,attempts
0,https://libras.cin.ufpe.br/sign/700,success,[https://libras.cin.ufpe.br/storage/videos/202...,"[3, 1, 2]",312,1
1,https://libras.cin.ufpe.br/sign/990,success,[https://libras.cin.ufpe.br/storage/videos/202...,"[3, 1, 2]",312,1
2,https://libras.cin.ufpe.br/sign/438,success,[https://libras.cin.ufpe.br/storage/videos/202...,"[1, 2, 3]",123,1
3,https://libras.cin.ufpe.br/sign/77,success,[https://libras.cin.ufpe.br/storage/videos/202...,"[1, 2, 3]",123,1
4,https://libras.cin.ufpe.br/sign/551,success,[https://libras.cin.ufpe.br/storage/videos/202...,"[1, 2, 3]",123,1


In [11]:
results_df.status.value_counts()

status
success    1364
Name: count, dtype: int64

In [12]:
results_df = results_df.explode(['video_links', 'signer_numbers'])
results_df.head()

,url,status,video_links,signer_numbers,signer_order,attempts
0,https://libras.cin.ufpe.br/sign/700,success,https://libras.cin.ufpe.br/storage/videos/2021...,3,312,1
0,https://libras.cin.ufpe.br/sign/700,success,https://libras.cin.ufpe.br/storage/videos/2021...,1,312,1
0,https://libras.cin.ufpe.br/sign/700,success,https://libras.cin.ufpe.br/storage/videos/2021...,2,312,1
1,https://libras.cin.ufpe.br/sign/990,success,https://libras.cin.ufpe.br/storage/videos/2021...,3,312,1
1,https://libras.cin.ufpe.br/sign/990,success,https://libras.cin.ufpe.br/storage/videos/2021...,1,312,1


In [13]:
words_and_links.head()

,label,sign_url
0,À noite toda,https://libras.cin.ufpe.br/sign/885
1,À tarde toda,https://libras.cin.ufpe.br/sign/100
2,Abacaxi,https://libras.cin.ufpe.br/sign/817
3,Abanar,https://libras.cin.ufpe.br/sign/1536
4,Abandonar,https://libras.cin.ufpe.br/sign/71


In [14]:
# join back to words_and_links on sign_url
results_df.rename(columns={'url':'sign_url', 'video_links':'video_url', 'signer_numbers':'signer_number'}, inplace=True)
metadata_df = words_and_links.merge(results_df, on='sign_url', how='left')
metadata_df.head()

,label,sign_url,status,video_url,signer_number,signer_order,attempts
0,À noite toda,https://libras.cin.ufpe.br/sign/885,success,https://libras.cin.ufpe.br/storage/videos/2021...,1,132,1
1,À noite toda,https://libras.cin.ufpe.br/sign/885,success,https://libras.cin.ufpe.br/storage/videos/2021...,3,132,1
2,À noite toda,https://libras.cin.ufpe.br/sign/885,success,https://libras.cin.ufpe.br/storage/videos/2021...,2,132,1
3,À tarde toda,https://libras.cin.ufpe.br/sign/100,success,https://libras.cin.ufpe.br/storage/videos/2020...,1,123,1
4,À tarde toda,https://libras.cin.ufpe.br/sign/100,success,https://libras.cin.ufpe.br/storage/videos/2020...,2,123,1


In [15]:
metadata_df.columns

Index(['label', 'sign_url', 'status', 'video_url', 'signer_number',
       'signer_order', 'attempts'],
      dtype='object')

In [ ]:
metadata_df['data_source'] = 'vl'

Save selected columns

In [16]:
metadata_df[
    ['label', 'video_url', 'signer_number', 'data_source', 'sign_url', 'signer_order']
       ].to_csv('data/raw/V-Librasil/metadata.csv', index=False)

## SignBank

### Explore the `full_metadata_SignBank.csv` file

In [165]:
sb_csv = pd.read_csv('data/raw/SignBank/full_metadata_SignBank.csv')
sb_csv.rename(columns={'Alphabet Label':'letter', 'Sign Label':'label', 'Video URL':'video_url'}, inplace=True)
sb_csv.head()

,letter,label,video_url
0,A,À-TOA,https://levantelab.storage.googleapis.com/libr...
1,A,À-VISTA,https://levantelab.storage.googleapis.com/libr...
2,A,ABACAXI,https://levantelab.storage.googleapis.com/libr...
3,A,ABANAR,https://levantelab.storage.googleapis.com/libr...
4,A,ABANDONAR,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...


In [166]:
sb_csv['video_url_root'] = sb_csv['video_url'].str.split('https://').str[1].str.split('.').str[0]
sb_csv['video_url_root'].value_counts()

video_url_root
videos        2992
objetos         63
levantelab      28
Name: count, dtype: int64

In [167]:
sb_csv['video_url_ext'] = sb_csv['video_url'].str.split('.').str[-1]
sb_csv['video_url_ext'].value_counts()

video_url_ext
mp4              3062
mov                21
URL_NOT_FOUND       1
Name: count, dtype: int64

In [168]:
sb_csv[sb_csv['video_url_ext'] == 'mov'].video_url_root.value_counts()

video_url_root
objetos    21
Name: count, dtype: int64

In [169]:
sb_csv[sb_csv['video_url_root'] == 'objetos'].video_url_ext.value_counts()

video_url_ext
mp4    42
mov    21
Name: count, dtype: int64

In [170]:
for i in [4,3,445]:
    print(sb_csv.iloc[i]['label'])
    print(sb_csv.iloc[i]['video_url_root'])
    print(sb_csv.iloc[i]['video_url'])
    print()

ABANDONAR
videos
https://videos.nals.cce.ufsc.br/SignBank/V%C3%ADdeos/ABANDONAR.mp4

ABANAR
levantelab
https://levantelab.storage.googleapis.com/libras/gerais/swiQhJZ6FoTLXBi1u8AgLBLjvhzUO1fbPLK1EEao.mp4

BOMBOM-2
objetos
https://objetos.sites.ufsc.br/libras/gerais/BaJ58VuGYbwzgx40LL8JhLfHLCAHVT2Uv4DLeXVt.mp4



### video_url validity (based on manual checking of a sample)

#### *videos.nals* - 2992 entries

URLs starting with `videos.nals...` are valid.

They correspond to videos from a signer with:
- facial hair
- an earring
- a black button-up shirt

They follow the URL format:
>`https://videos.nals.cce.ufsc.br/SignBank/Vídeos/{label}.mp4`

The web URL can handle accented letters. The í in Videos is accented. The {label} is sometimes accented.

#### *objetos* - 63 entries

URLs starting with `objetos...` are valid.

They correspond to videos from a signer with:
- no facial hair
- a black t-shirt

They follow the URL format:
`https://objetos.sites.ufsc.br/libras/gerais/{40-character-code}.{file-extension}`
- where `40-character-code` is a 40 character code (possibly derived from the label?) like:
    - `HWQBq7whvIWSxzN3HHii6xk0Npu0qUTxJBZiMPeI`
    - `I021pwWFtHqpgspqKCpiS5Q8quymB9urO9wfJCru`
- and `file-extension` is either `.mp4` or `.mov`
    - 42 end in `.mp4`, which opens the video in browser
    - 21 end in `.mov`, which starts a direct download


#### *levantelab* - 28 entries
URLs starting with `levantelab.storage...` are invalid and return:
>This XML file does not appear to have any style information associated with it. The document tree is shown below.
>```
><Error>
><Code>UserProjectAccountProblem</Code>
><Message>The project to be billed is associated with a closed billing account.</Message>
><Details>The billing account for the owning project is disabled in state closed</Details>
></Error>
>```

The videos don't load on the SignBank portal either, so the issue is on their end. 

They follow the URL format:
`https://levantelab.storage.googleapis.com/libras/gerais/{40-character-code}.mp4`

Copying the `{40-character-code}` into the *objetos* URL format **DOES NOT** work:
>...\
>```<Message>The specified key does not exist.</Message>```\
>...

Copying the label into the *videos.nals* URL format **DOES** work:
- ABANAR: https://levantelab.storage.googleapis.com/libras/gerais/swiQhJZ6FoTLXBi1u8AgLBLjvhzUO1fbPLK1EEao.mp4
    - `INVALID`
- ABANAR: https://videos.nals.cce.ufsc.br/SignBank/Vídeos/{ABANAR}.mp4
    - `VALID`

### Fixing URLs

In [171]:
sb_csv.rename(columns={'video_url':'scraped_url'}, inplace=True)
sb_csv['fixed_url'] = sb_csv['scraped_url']

Something happened with `FOLGAR`, as the video on the SignBank portal is availabe, and the URL `https://videos.nals.cce.ufsc.br/SignBank/Vídeos/FOLGAR.mp4` works

In [172]:
sb_csv[sb_csv['video_url_ext'] == 'URL_NOT_FOUND']

,letter,label,scraped_url,video_url_root,video_url_ext,fixed_url
1248,F,FOLGAR,URL_NOT_FOUND,NaN,URL_NOT_FOUND,URL_NOT_FOUND


In [173]:
# manually change entries at label == 'FOLGAR'
index = sb_csv[sb_csv['label'] == 'FOLGAR'].index
sb_csv.loc[index, 'fixed_url'] = 'https://videos.nals.cce.ufsc.br/SignBank/Vídeos/FOLGAR.mp4'
sb_csv.loc[index,'video_url_root'] = 'videos'
sb_csv.loc[index,'video_url_ext'] = 'mp4'

In [174]:
sb_csv[sb_csv['label'] == 'FOLGAR']

,letter,label,scraped_url,video_url_root,video_url_ext,fixed_url
1248,F,FOLGAR,URL_NOT_FOUND,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/Vídeo...


Fix the 28 `levantelab` entries

In [175]:
sb_csv[sb_csv['video_url_root'] == 'levantelab']

,letter,label,scraped_url,video_url_root,video_url_ext,fixed_url
0,A,À-TOA,https://levantelab.storage.googleapis.com/libr...,levantelab,mp4,https://levantelab.storage.googleapis.com/libr...
1,A,À-VISTA,https://levantelab.storage.googleapis.com/libr...,levantelab,mp4,https://levantelab.storage.googleapis.com/libr...
2,A,ABACAXI,https://levantelab.storage.googleapis.com/libr...,levantelab,mp4,https://levantelab.storage.googleapis.com/libr...
3,A,ABANAR,https://levantelab.storage.googleapis.com/libr...,levantelab,mp4,https://levantelab.storage.googleapis.com/libr...
5,A,ABELHA,https://levantelab.storage.googleapis.com/libr...,levantelab,mp4,https://levantelab.storage.googleapis.com/libr...
6,A,ABENÇOAR,https://levantelab.storage.googleapis.com/libr...,levantelab,mp4,https://levantelab.storage.googleapis.com/libr...
7,A,ABORDAGEM,https://levantelab.storage.googleapis.com/libr...,levantelab,mp4,https://levantelab.storage.googleapis.com/libr...
10,A,ABRAÇAR,https://levantelab.storage.googleapis.com/libr...,levantelab,mp4,https://levantelab.storage.googleapis.com/libr...
96,A,ALEMANHA,https://levantelab.storage.googleapis.com/libr...,levantelab,mp4,https://levantelab.storage.googleapis.com/libr...
284,A,ABRIR,https://levantelab.storage.googleapis.com/libr...,levantelab,mp4,https://levantelab.storage.googleapis.com/libr...


Sometimes there is multiple videos for one label.

In [176]:
def number_in_label(label):
    for char in label:
        if char.isdigit():
            return True
    return False
sb_csv['number_in_label'] = sb_csv['label'].apply(number_in_label)
sb_csv['number_in_label'].value_counts()

number_in_label
False    2509
True      575
Name: count, dtype: int64

In the label, these are denoted with:
- {label}-1
- {label}-2

In the URL, these are denoted with:
- /{label}1
- /{label}2

In [177]:
n = 5
for i, row in sb_csv[sb_csv['number_in_label'] == True].sample(n).iterrows():
    print(row['label'])
    print(row['scraped_url'])
    print()


BRÓCOLIS-2
https://videos.nals.cce.ufsc.br/SignBank/V%C3%ADdeos/BR%C3%93COLIS2.mp4

YOUTUBE-2
https://videos.nals.cce.ufsc.br/SignBank/V%C3%ADdeos/YOUTUBE2.mp4

BATISTA-2
https://videos.nals.cce.ufsc.br/SignBank/V%C3%ADdeos/BATISTA2.mp4

RÁPIDO-2
https://videos.nals.cce.ufsc.br/SignBank/V%C3%ADdeos/R%C3%81PIDO2.mp4

ALFACE-2
https://videos.nals.cce.ufsc.br/SignBank/V%C3%ADdeos/ALFACE2.mp4



Remake levantelab URLs

In [178]:
def remake_levantelab_url(label):
    if number_in_label(label):
        # ESTADOS-UNIDOS-DA-AMÉRICA-2  -> ESTADOS-UNIDOS-DA-AMÉRICA2
        label = label[:-2] + label[-1]
    url = f"https://videos.nals.cce.ufsc.br/SignBank/Vídeos/{label}.mp4"
    return url

remake_levantelab_url("ESTADOS-UNIDOS-DA-AMÉRICA-2")

'https://videos.nals.cce.ufsc.br/SignBank/Vídeos/ESTADOS-UNIDOS-DA-AMÉRICA2.mp4'

In [179]:
# apply remake_levantelab_url to the levantelab entries
sb_csv.loc[sb_csv['video_url_root'] == 'levantelab', 'fixed_url'] = sb_csv.loc[sb_csv['video_url_root'] == 'levantelab', 'label'].apply(remake_levantelab_url)

In [180]:
n = 5
for i, row in sb_csv[sb_csv['video_url_root'] == 'levantelab'].sample(n).iterrows():
    print(row['label'])
    print(row['scraped_url'])
    print(row['fixed_url'])
    print()

VEGETARIANO
https://levantelab.storage.googleapis.com/libras/gerais/fUObXisMnCnMxh2MHGUHukqDYiz1BhHuVBMCbxOG.mp4
https://videos.nals.cce.ufsc.br/SignBank/Vídeos/VEGETARIANO.mp4

ESTADOS-UNIDOS-DA-AMÉRICA-2
https://levantelab.storage.googleapis.com/libras/gerais/L66RywqgLQWoYLuetwHLLostRXpQCLgfJKRD5T7m.mp4
https://videos.nals.cce.ufsc.br/SignBank/Vídeos/ESTADOS-UNIDOS-DA-AMÉRICA2.mp4

ABACAXI-2
https://levantelab.storage.googleapis.com/libras/gerais/h7ZOyw1GrUWJXxFwU8oJ997sAQE2SY3oyJ9hxI2s.mp4
https://videos.nals.cce.ufsc.br/SignBank/Vídeos/ABACAXI2.mp4

ACUSAR
https://levantelab.storage.googleapis.com/libras/gerais/SSQokxkM9jDJWLIJrRVg1SlnehJD6jq4cFE1c4OJ.mp4
https://videos.nals.cce.ufsc.br/SignBank/Vídeos/ACUSAR.mp4

ABRIR
https://levantelab.storage.googleapis.com/libras/gerais/GIPQ5FwLKp0k9XdAk5ajTtR13qVuxYdqzfsg83xV.mp4
https://videos.nals.cce.ufsc.br/SignBank/Vídeos/ABRIR.mp4



### Make `clean_full_metadata_SignBank.csv`

#### Combine labels with numbers
- usually a dash on labels with numbers, e.g. INCLUSÃO-3
- sometimes no dash, e.g. PARA-FRENTE2

In [181]:
sb_csv['scraped_label'] = sb_csv['label']
sb_csv['sign_variant'] = 1

In [182]:
sb_csv[sb_csv['number_in_label'] == True].sample(5)

,letter,label,scraped_url,video_url_root,video_url_ext,fixed_url,number_in_label,scraped_label,sign_variant
1709,M,MÃE-3,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,MÃE-3,1
846,C,CHAMAR-3,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,CHAMAR-3,1
1813,M,MESMO-4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,MESMO-4,1
1985,O,OITO-2,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,OITO-2,1
2980,V,VERDE-3,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,VERDE-3,1


In [183]:
def remove_number_from_label(label):
    # usually a dash on labels with numbers, e.g. INCLUSÃO-3
    # sometimes no dash, e.g. PARA-FRENTE2

    # go from right to left and remove the number
    if label[-1].isdigit():
        label = label[:-1]
    else:
        print(f"no number at the end of '{label}'")
        return
    if label[-1] == '-':
        label = label[:-1]
    return label   

remove_number_from_label("INCLUSÃO-3"), remove_number_from_label("PARA-FRENTE2"), remove_number_from_label("ESTADOS-UNIDOS-DA-AMÉRICA-2"), remove_number_from_label("no-number-test-")

no number at the end of 'no-number-test-'


('INCLUSÃO', 'PARA-FRENTE', 'ESTADOS-UNIDOS-DA-AMÉRICA', None)

In [184]:
sb_csv.loc[sb_csv['number_in_label'] == True, 'sign_variant'] = sb_csv.loc[sb_csv['number_in_label'] == True, 'scraped_label'].str[-1].astype(int)

In [185]:
sb_csv.loc[sb_csv['number_in_label'] == True, 'label'] = sb_csv.loc[sb_csv['number_in_label'] == True, 'scraped_label'].apply(remove_number_from_label)

In [186]:
sb_csv[sb_csv['number_in_label'] == True].sample(5)

,letter,label,scraped_url,video_url_root,video_url_ext,fixed_url,number_in_label,scraped_label,sign_variant
843,C,COMER,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,COMER-3,3
2138,P,PASTA,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,PASTA-2,2
1475,I,INCLUSÃO,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,INCLUSÃO-4,4
1572,J,JORNAL,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,JORNAL-2,2
1033,E,ELEVADOR,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,ELEVADOR-2,2


Add a `signer_number` col

In [187]:
sb_csv['signer_number'] = 1
# based on an assumption that the videos at `videos.nals...` have the main signer, and `levantelab` have the other one
sb_csv.loc[sb_csv['video_url_root'] == 'levantelab', 'signer_number'] = 2

#### Checking any exceptions in the processing

In [188]:
sb_csv[sb_csv.duplicated(subset=['label', 'sign_variant', 'signer_number'], keep=False)][['label', 'scraped_label', 'sign_variant', 'number_in_label']].sort_values(by='label')

,label,scraped_label,sign_variant,number_in_label
1199,FERRO,FERRO,1,False
1298,FERRO,FERRO,1,False
1472,INCLUSÃO,INCLUSÃO,1,False
1477,INCLUSÃO,INCLUSÃO-1,1,True
2957,VAZIO,VAZIO-2,2,True
3056,VAZIO,VAZIO-2,2,True
3028,VITÓRIA,VITÓRIA,1,False
3029,VITÓRIA,VITÓRIA,1,False


- FERRO
    - Two videos of the same sign for FERRO
    - Not a sign_variant
    - -> Just label different signers
- INCLUSÃO
    - has 6 different sign_variant
    - but labelled differently by SignBank (first variant labelled 1 instead of 2)
    - -> change INCLUSÃO sign_variant to 6
- VAZIO
    - Two videos of the same sign for VAZIO-2
    - Not a sign_variant
    - -> Just label different signers
- VITÓRIA
    - Two videos of different signs for VITÓRIA
    - -> Label one as a sign_variant


Unchecked assumption is that the videos at `videos.nals...` have the main signer, and `levantelab` have the other one`

Ferro

In [189]:
sb_csv.loc[1298, 'signer_number'] = 2

Inclusao

In [190]:
sb_csv.loc[sb_csv['label'] == 'INCLUSÃO']

,letter,label,scraped_url,video_url_root,video_url_ext,fixed_url,number_in_label,scraped_label,sign_variant,signer_number
1472,I,INCLUSÃO,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,False,INCLUSÃO,1,1
1473,I,INCLUSÃO,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,INCLUSÃO-5,5,1
1474,I,INCLUSÃO,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,INCLUSÃO-2,2,1
1475,I,INCLUSÃO,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,INCLUSÃO-4,4,1
1476,I,INCLUSÃO,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,INCLUSÃO-3,3,1
1477,I,INCLUSÃO,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,INCLUSÃO-1,1,1


In [191]:
sb_csv.loc[1472, 'sign_variant'] = 6

Vazio

In [192]:
sb_csv.loc[sb_csv['scraped_label'] == 'VAZIO-2']

,letter,label,scraped_url,video_url_root,video_url_ext,fixed_url,number_in_label,scraped_label,sign_variant,signer_number
2957,V,VAZIO,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,VAZIO-2,2,1
3056,V,VAZIO,https://objetos.sites.ufsc.br/libras/gerais/lI...,objetos,mov,https://objetos.sites.ufsc.br/libras/gerais/lI...,True,VAZIO-2,2,1


In [193]:
sb_csv.loc[sb_csv['scraped_label'] == 'VAZIO-2'].scraped_url.values

array(['https://videos.nals.cce.ufsc.br/SignBank/V%C3%ADdeos/VAZIO2.mp4',
       'https://objetos.sites.ufsc.br/libras/gerais/lImvYVGMUUeodhH3TuKWk0s2SBGe414Buvi18NsU.mov'],
      dtype=object)

In [194]:
sb_csv.loc[3056, 'signer_number'] = 2

Vitoria

In [195]:
sb_csv.loc[sb_csv['scraped_label'] == 'VITÓRIA']

,letter,label,scraped_url,video_url_root,video_url_ext,fixed_url,number_in_label,scraped_label,sign_variant,signer_number
3028,V,VITÓRIA,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,False,VITÓRIA,1,1
3029,V,VITÓRIA,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,False,VITÓRIA,1,1


In [196]:
sb_csv.loc[sb_csv['scraped_label'] == 'VITÓRIA'].scraped_url.values

array(['https://videos.nals.cce.ufsc.br/SignBank/V%C3%ADdeos/VIT%C3%93RIA.mp4',
       'https://videos.nals.cce.ufsc.br/SignBank/V%C3%ADdeos/VIT%C3%93RIA-ES.mp4'],
      dtype=object)

In [197]:
sb_csv.loc[3029, 'sign_variant'] = 2

Check the duplicates are fixed

In [198]:
sb_csv[sb_csv.duplicated(subset=['label', 'sign_variant', 'signer_number'], keep=False)][['label', 'scraped_label', 'sign_variant', 'number_in_label', 'signer_number']].sort_values(by='label')

,label,scraped_label,sign_variant,number_in_label,signer_number


#### Finalise `clean_full_metadata_SignBank.csv` and save

In [199]:
sb_csv.sample(5)

,letter,label,scraped_url,video_url_root,video_url_ext,fixed_url,number_in_label,scraped_label,sign_variant,signer_number
1875,M,MOSCA,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,False,MOSCA,1,1
775,C,COORDENADOR,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,COORDENADOR-2,2,1
2042,O,OUVIR,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,False,OUVIR,1,1
573,C,CATAPORA,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,False,CATAPORA,1,1
105,A,ALHO,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,videos,mp4,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,True,ALHO-2,2,1


In [212]:
sb_csv.columns

Index(['letter', 'label', 'scraped_url', 'video_url_root', 'video_url_ext',
       'fixed_url', 'number_in_label', 'scraped_label', 'sign_variant',
       'signer_number'],
      dtype='object')

In [213]:
sb_csv[[
    'label', 'scraped_label', 'fixed_url', 'scraped_url',
    'sign_variant', 'signer_number', 'video_url_ext', 'video_url_root',
    'number_in_label', 'letter'
]].to_csv('data/raw/SignBank/clean_full_metadata_SignBank.csv', index=False)

### make `metadata.csv`

In [16]:
clean_signbank = pd.read_csv('data/raw/SignBank/clean_full_metadata_SignBank.csv')

In [17]:
clean_signbank.head()

,label,scraped_label,fixed_url,scraped_url,sign_variant,signer_number,video_url_ext,video_url_root,number_in_label,letter
0,À-TOA,À-TOA,https://videos.nals.cce.ufsc.br/SignBank/Vídeo...,https://levantelab.storage.googleapis.com/libr...,1,2,mp4,levantelab,False,A
1,À-VISTA,À-VISTA,https://videos.nals.cce.ufsc.br/SignBank/Vídeo...,https://levantelab.storage.googleapis.com/libr...,1,2,mp4,levantelab,False,A
2,ABACAXI,ABACAXI,https://videos.nals.cce.ufsc.br/SignBank/Vídeo...,https://levantelab.storage.googleapis.com/libr...,1,2,mp4,levantelab,False,A
3,ABANAR,ABANAR,https://videos.nals.cce.ufsc.br/SignBank/Vídeo...,https://levantelab.storage.googleapis.com/libr...,1,2,mp4,levantelab,False,A
4,ABANDONAR,ABANDONAR,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,1,1,mp4,videos,False,A


In [18]:
clean_signbank.rename(columns={
    'fixed_url':'video_url',
    'scraped_url':'scraped_video_url',
    }, inplace=True)

In [19]:
clean_signbank['data_source'] = 'sb'

In [20]:
clean_signbank[[
    "label",
    "video_url",
    "signer_number",
    "data_source",
    "scraped_label",
    "scraped_video_url",
    "sign_variant",
    "signer_number",
    "video_url_root",
    "video_url_ext",
    "number_in_label",
]].to_csv('data/raw/SignBank/metadata.csv', index = False)

## UFV

To Do

# Combine into one `metadata_combined.csv`

In [4]:
ne_metadata = pd.read_csv('data/raw/INES/metadata.csv')
sb_metadata = pd.read_csv('data/raw/SignBank/metadata.csv')
vl_metadata = pd.read_csv('data/raw/V-Librasil/metadata.csv')

In [6]:
metadata_combined = pd.concat([ne_metadata, sb_metadata, vl_metadata])
metadata_combined.to_csv('data/raw/combined/metadata_combined_unclean.csv', index=False)